In [1]:
import h5py
import scipy.io as io
import PIL.Image as Image
import numpy as np
import os
import glob
from matplotlib import pyplot as plt
from scipy.ndimage.filters import gaussian_filter 
import scipy
import json
import torchvision.transforms.functional as F
from matplotlib import cm as CM
from image import *
from model import CSRNet
import torch
%matplotlib inline

In [2]:
from torchvision import datasets, transforms
transform=transforms.Compose([
                       transforms.ToTensor(),transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                     std=[0.229, 0.224, 0.225]),
                   ])

In [3]:
model = CSRNet()

Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/checkpoints/vgg16-397923af.pth
100%|██████████| 528M/528M [00:23<00:00, 24.0MB/s]


In [4]:
model = model.cuda()

In [5]:
WEIGHTS='/weights/0model_best.pth.tar'

In [6]:
checkpoint = torch.load(WEIGHTS)

In [7]:
model.load_state_dict(checkpoint['state_dict'])

IncompatibleKeys(missing_keys=[], unexpected_keys=[])

## Extract from video frame

In [9]:
!pip install moviepy

     |████████████████████████████████| 399kB 3.7MB/s 
     |████████████████████████████████| 3.3MB 6.5MB/s 
     |████████████████████████████████| 22.2MB 30.1MB/s 
     |████████████████████████████████| 2.1MB 43.1MB/s 
  Stored in directory: /root/.cache/pip/wheels/52/e2/4c/f594a5945bc98e052ef248b46a0f1f7ea838b0b2a5f8895651
  Stored in directory: /root/.cache/pip/wheels/65/56/60/1d0306a8d90b188af393c1812ddb502a8821b70917f82dcc00
Successfully built moviepy proglog


In [32]:
from moviepy.editor import VideoFileClip
from IPython.display import HTML
from collections import deque
import cv2

In [39]:
def process_image(image_nparray):
    draw_image = np.copy(image_nparray)
    # overhead here to convert np-array back to PIL IMAGE
    pil_img = Image.fromarray(image_nparray.astype('uint8'), 'RGB')
    img_tensor = transform(pil_img).cuda()
    pred = model(img_tensor.unsqueeze(0))
    pred_num = int(pred.detach().cpu().sum().numpy())
    cv2.putText(draw_image, 'pred_num= {}'.format(pred_num), (550, 50), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 0, 0), 2, cv2.LINE_AA)
    return draw_image

In [40]:
IN_VIDEO='/dataset/hk.mp4'
OUT_VIDEO='/dataset/hk_res_2.mp4'

In [41]:
clip = VideoFileClip(IN_VIDEO)
out_clip = clip.fl_image(process_image) 
%time out_clip.write_videofile(OUT_VIDEO, audio=False)

t:   0%|          | 0/5362 [00:00<?, ?it/s, now=None]

Moviepy - Building video /dataset/hk_res_2.mp4.
Moviepy - Writing video /dataset/hk_res_2.mp4



Moviepy - Done !
Moviepy - video ready /dataset/hk_res_2.mp4
CPU times: user 21min 25s, sys: 7min 23s, total: 28min 48s
Wall time: 14min 54s
